# The Training Process
So, we are almost done preparing our dataset and I feel as though it's a good time to explain how our model is trained. Specifically, how input data is fed to our model.

For this specific model data is going to be streamed into it in small batches of 32. This means we will not feed the entire dataset to our model at once, but simply small batches of entries. We will feed these batches to our model multiple times according to the number of epochs.

An epoch is simply one stream of our entire dataset. The number of epochs we define is the amount of times our model will see the entire dataset. We use multiple epochs in hope that after seeing the same data multiple times the model will better determine how to estimate it.

Ex. if we have 10 ephocs, our model will see the same dataset 10 times.

Since we need to feed our data in batches and multiple times, we need to create something called an input function. The input function simply defines how our dataset will be converted into batches at each epoch.

## Input Function
The TensorFlow model we are going to use requires that the data we pass it comes in as a tf.data.Dataset object. This means we must create a input function that can convert our current pandas dataframe into that object.

Below you'll see a seemingly complicated input function, this is straight from the TensorFlow documentation (<a>https://www.tensorflow.org/tutorials/estimator/linear</a>). I've commented as much as I can to make it understandble, but you may want to refer to the documentation for a detailed explination of each method.

Now first let us input the data from dataset.

In [27]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow as tf
import tensorflow.compat.v1 as fc
from tensorflow.keras.layers import StringLookup, IntegerLookup, Normalization
from tensorflow.keras import layers

# Example TensorFlow usage
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.17.0


# Load the data

In [28]:
# Load the data
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

# Convert categorical columns to string type to avoid any type issues
dftrain[CATEGORICAL_COLUMNS] = dftrain[CATEGORICAL_COLUMNS].astype(str)
dfeval[CATEGORICAL_COLUMNS] = dfeval[CATEGORICAL_COLUMNS].astype(str)

# Preprocess data

In [30]:
# Preprocessing layers
preprocessing_layers = {}

# Create StringLookup layers for categorical features
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique().tolist()
  preprocessing_layers[feature_name] = StringLookup(vocabulary=vocabulary, output_mode='int')

# Create Normalization layers for numeric features
for feature_name in NUMERIC_COLUMNS:
  normalizer = Normalization()
  normalizer.adapt(dftrain[feature_name].values)
  preprocessing_layers[feature_name] = normalizer

# Function to preprocess the dataset
def preprocess(features, label):
  numeric_features = {}
  for feature_name in NUMERIC_COLUMNS:
    numeric_features[feature_name] = preprocessing_layers[feature_name](features[feature_name])
  return numeric_features, label

# Convert the DataFrame to a TensorFlow Dataset
train_ds = tf.data.Dataset.from_tensor_slices((dict(dftrain), y_train))
eval_ds = tf.data.Dataset.from_tensor_slices((dict(dfeval), y_eval))

# Apply the preprocessing function to the datasets
train_ds = train_ds.map(preprocess)
eval_ds = eval_ds.map(preprocess).batch(1)

# Batch the dataset
train_ds = train_ds.batch(32)

## Creating model

In [31]:
# Define a simple model
def first_layer(inputs):
  age = inputs['age']
  fare = inputs['fare']
  # Check shapes (optional)
  print(f"age shape: {age.shape}")
  print(f"fare shape: {fare.shape}")
  # Reshape if necessary ...
  combined_features = tf.concat([age, fare], axis=-1)
  return combined_features

model = tf.keras.Sequential([
  layers.Lambda(first_layer),  # Custom layer with input dictionary
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model

In [32]:
# Train the model
model.fit(train_ds, epochs=10)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5475 - loss: 144496208.0000   
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5644 - loss: 56208244.0000 
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5612 - loss: 132835656.0000 
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5860 - loss: 24223588.0000 
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5318 - loss: 25444878.0000 
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5388 - loss: 4310463.0000 
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5638 - loss: 3880836.2500 
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5621 - loss: 3622376.0000 
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5531 - loss: 4003047.2500 
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5154 - loss: 5352905.5000 


# Evaluate the model

In [33]:
loss, accuracy = model.evaluate(eval_ds)
print('Test accuracy:', accuracy)
print("Model loss: ", loss)

264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5732 - loss: 8902684.0000
Test accuracy: 0.6553030014038086
Model loss:  6704605.0


# Predict


In [56]:
result = list(model.predict(eval_ds))
print(result)

  1/264 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step

264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step
[array([0.], dtype=float32), array([0.50199735], dtype=float32), array([0.50199735], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.50199735], dtype=float32), array([0.50199735], dtype=float32), array([0.], dtype=float32), array([0.50199735], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.50199735], dtype=float32), array([0.50199735], dtype=float32), array([0.50199735], dtype=float32), array([1.], dtype=float32), array([0.50199735], dtype=float32), array([0.], dtype=float32), array([0.5019973